## 「[ゼロから作るDeep Learning ❸――フレームワーク編](https://www.oreilly.co.jp/books/9784873119069/)」を元にPythonの基礎解説2
この記事では、前回に続けてPythonの基礎解説と雛形の作成を行なっていきます。DeZeroやNumPyについては詳しい説明は行いません。

### Pythonの参考文献
 - [Pythonのlambda（ラムダ式、無名関数）の使い方 | note.nkmk.me](https://note.nkmk.me/python-lambda-usage/)
 - 外部モジュール[memory-profiler](https://pypi.org/project/memory-profiler/)
 - [unittest --- ユニットテストフレームワーク](https://docs.python.org/ja/3/library/unittest.html)
 - [2. setup スクリプトを書く - Python 3 ドキュメント](https://docs.python.org/ja/3/distutils/setupscript.html)

## 「第2ステージ 自然なコードで表現する」より
「第2ステージ」で、DeZeroは可変長の入出力を実現し、世代の概念が実装され、多くの変数・演算子が自然に扱えるようになり、パッケージにまとめられます。

### 「ステップ11 可変長の引数（順伝播編）」より
 - Pythonのリストは、`[1, 2 ,3]`のように`[]`で囲み、タプルは`(1, 2, 3)`のように`()`で囲み複数の値を保持する。タプルは一度生成したら、その要素を変更できない。
 - リスト内包表記は、`[c.velocity for c in cars]`のように書き、`cars`リストの各要素`c`に対して、それぞれのデータ`c.velocity`を取り出し、その要素からなる新しいリストを作る。

In [1]:
class Vehicle:
    def __init__(self, velocity):
        if velocity is not None:
            if not isinstance(velocity, int):
                raise TypeError('{} is not supported'.format(type(velocity)))
        self.velocity = velocity

    def __call__(self, vehicle):
        self.velocity += vehicle.velocity
        return self 

    def run(self):
        raise NotImplementedError()


class Car(Vehicle):
    def run(self):
        print('Run at {}.'.format(self.velocity))
        self.velocity -= 1
        if self.velocity > 0:
            self.run()

cars = [Car(7), Car(5), Car(3)]
car_velocities = [c.velocity for c in cars]
print(car_velocities)

[7, 5, 3]


### 「ステップ12 可変長の引数（改善編）」より
 - Pythonのif文を一行で記述する三項演算子（条件演算子）。
   - `条件式が真のときに評価される式 if 条件式 else 条件式が偽のときに評価される式`
 - 関数定義の引数にアスタリスク`*`を付けることにより、任意個の引数（可変長引数）を与えられる。
 - 関数の呼び出し時にアスタリスク`*`を付けることで、リストの要素を展開して渡すアンパックが行われる。

In [2]:
a = 1
print('even' if a % 2 == 0 else 'odd') # 三項演算子（値）
a = 2
print('even') if a % 2 == 0 else print('odd') # 三項演算子（式）

def f(*x): # 可変長引数
    print(type(x))
    print(x)
 
f(1,2,3)
f(1)

p = [1, 2]
print(p)
print(*p) # アンパック
print(3, 4)

f(p)
f(*p) # アンパック

odd
even
<class 'tuple'>
(1, 2, 3)
<class 'tuple'>
(1,)
[1, 2]
1 2
3 4
<class 'tuple'>
([1, 2],)
<class 'tuple'>
(1, 2)


### 「ステップ13 可変長の引数（逆伝播編）」より
 - `zip`関数で配列のペアから値を取得する。

In [3]:
for c, n in zip(["a", "b", "c", "d"], [1,2,3]):
    print(c, n)

a 1
b 2
c 3


### 「ステップ14 同じ変数を繰り返し使う」
### 「ステップ15 複雑な計算グラフ（理論編）」
### 「ステップ16 複雑な計算グラフ（実装編）」より
 - 配列（リスト、タプル、集合、辞書）から最大値を取り出す関数`max`
 - ラムダ式（無名関数）の文法：`lambda 引数1, 引数2, ...: 返式`
   - 次の関数定義は同等であり、名前を付けなくても良い。返式には三項演算子を使ってもいい。`sorted`、`max`などの関数の`key`引数に渡す関数としてよく使われる。`map`関数や`filter`関数のような第２引数のリストなどに適用する第１引数の関数としてもよく使われる。

```python
def func(a, b):
    return a + b

func = lambda a, b: a + b
```

 - 参考：[Pythonのlambda（ラムダ式、無名関数）の使い方 | note.nkmk.me](https://note.nkmk.me/python-lambda-usage/)

In [4]:
print(max(1, 2, 3), max((1, 2, 3)), max([1, 2, 3]), max({1, 2, 3}))
dictionary = {'a': 3, 'b': 2, 'c': 1}
print(max(dictionary)) # 辞書のキーの最大
print(dictionary.items())
print(max(dictionary.items(), key = lambda x: x[1])) # 辞書の値が最大の項取得
print((lambda a: 'even' if a % 2 == 0 else 'odd')(7)) # 三項演算子

3 3 3 3
c
dict_items([('a', 3), ('b', 2), ('c', 1)])
('a', 3)
odd


### 「ステップ17 メモリ管理と循環参照」より
 - Pythonのメモリ管理は、参照カウントとガーベージコレクション（GC）と呼ばれる方式で行われている。
 - 参照カウント方式では、オブジェクトは参照カウントが0の状態で生成され、他のオブジェクトから参照されたときに、参照カウントを+1し、参照がなくなると、参照カウントを-1し、参照カウンタが0になったタイミングで、オブジェクトは即座にメモリから消去される。
 - 次の様にオブジェクト`a, b`がそれぞれオブジェクト`b, a`を循環参照している場合は、`a = b = None`後はプログラムから`a, b`オブジェクトは利用できないが参照カウントは0にならずメモリから消去されない。
   - この場合はメモリが不足したタイミングなどでGCによりメモリから解放されるがメモリ解放の先送りになるためメモリ使用量は増加する。
   - `gc`モジュールをインポートし`gc.collect()`によりGCを明示的に呼び出すこともできる。

```python
class Obj:
    pass

a = Obj() # オブジェクトaの参照カウント1
b = Obj() # オブジェクトbの参照カウント1

a.b = b # オブジェクトbの参照カウント2
b.a = a # オブジェクトaの参照カウント2（循環参照）
a = b = None # オブジェクトa, bの参照カウント1
```

 - `weakref`モジュールの`weakref.ref`関数を使って、参照カウントを増やすことなく、別のオブジェクトを参照する弱参照を行える。弱参照先のオブジェクトの値にアクセスするには`()`を付ける。
 - 外部ライブラリの[memory-profiler](https://pypi.org/project/memory-profiler/)を使えば、メモリ使用量が計測できる。

In [5]:
import weakref
import numpy as np

a = np.array([1, 2, 3]) # オブジェクトaの参照カウント1
b = weakref.ref(a) # オブジェクトaの参照カウント1のまま
print(b)
print(b()) # オブジェクトaの値にアクセス
a = None # オブジェクトaの参照カウント0（メモリ解放）
print(b)

<weakref at 0x7fd5e4935230; to 'numpy.ndarray' at 0x7fd5e48ef2b0>
[1 2 3]
<weakref at 0x7fd5e4935230; dead>


### 「ステップ 18 メモリ使用量を減らすモード」より
 - `with`文により、`with`ブロックに入るときの前処理、`with`ブロックから出るときの後処理を自動的に行える。
 - `contextlib`モジュールを使い、関数定義に`@contextlib.contextmanager`デコレータを付けることで`with`文を実装できる。

In [6]:
import contextlib

@contextlib.contextmanager
def with_test():
    print('前処理')
    try:
        yield
    finally:
        print('後処理')

with with_test():
    print('withブロック内の処理')

前処理
withブロック内の処理
後処理


### 「ステップ19 変数を使いやすく」より
 - 関数定義の前に`@property`デコレータを付けることで、関数はインスタンス変数としてアクセスできるようになる。
 - `len`関数は要素数を返す。高次元配列は最初の次元の要素数を返す。
 - クラスに`__len__`特殊メソッドを実装することで、そのクラスのインスタンスに対しても`len`関数が使えるようになる。
 - クラスに`__repr__`特殊メソッドを実装することで、そのクラスのインスタンスに対する`str`関数、`print`関数をカスタマイズできる。クラスに`__str__`特殊メソッドを実装することによっても`str`関数をカスタマイズできる。

In [7]:
class Vehicle:
    def __init__(self, velocity):
        self.velocity = velocity

    def __call__(self, velocity):
        self.velocity += velocity
        
    def __len__(self): # len関数
        return self.velocity
    
    def __repr__(self):
        return 'Velocity is {}.'.format(self.velocity)

    def run(self):
        raise NotImplementedError()

class Car(Vehicle):
    @property
    def run(self):
        print('Run at {}.'.format(self.velocity))

car = Car(10)
car.run # インスタンス変数として

print(len([1, 2, 3, 4]), len([[1, 2, 3], [4, 5, 6]]))
print(len(np.array([1, 2, 3, 4])), len(np.array([[1, 2, 3], [4, 5, 6]])))
print(len(car)) # __len__
print(car) # __repr__
print(str(car))

Run at 10.
4 2
4 2
10
Velocity is 10.
Velocity is 10.


### 「ステップ20 演算子のオーバーロード（1）」より
### 「ステップ21 演算子のオーバーロード（2）」より
### 「ステップ22 演算子のオーバーロード（3）」より
 - Pythonでは、次表のような特殊メソッドを定義することで、対応する演算子をオーバーロードし、ユーザが定義した処理を呼べるようになる。演算子は表の他にもある。
   - `__op__`は`__rop__`より優先度が高く、`__op__`が実装されてない場合に`__rop__`が呼ばれる。

|特殊メソッド                 |演算子          |
|:--------------------------|:--------------|
|`__add__(self, other)`     |`self + other` |
|`__radd__(self, other)`    |`other + self` |
|`__mul__(self, other)`     |`self * other` |
|`__rmul__(self, other)`    |`other * self` |
|`__neg__(self)`            |`-self`        |
|`__sub__(self, other)`     |`self - other` |
|`__rsub__(self, other)`    |`other - self` |
|`__truediv__(self, other)` |`self / other` |
|`__rtruediv__(self, other)`|`other / self` |
|`__pow__(self, other)`     |`self ** other`|

- 演算子のオーバーロードは別に定義した関数オブジェクトを用いて`Vehicle.__add__ = add`のように実装することもできる。

In [8]:
class Vehicle:
    def __init__(self, velocity):
        self.velocity = velocity

    def __call__(self, velocity):
        self.velocity += velocity
        
    def __len__(self): # len関数
        return self.velocity
    
    def __repr__(self):
        return 'Velocity is {}.'.format(self.velocity)
    
    def __add__(self, other):
        if isinstance(other, Vehicle):
            return Vehicle(self.velocity + other.velocity)
        elif isinstance(other, int):
            return Vehicle(self.velocity + other)
        else:
            raise TypeError()
    
    def __radd__(self, other):
        if isinstance(other, int):
            return Vehicle(other + self.velocity)
        else:
            raise TypeError()
    
    def __mul__(self, other):
        if isinstance(other, Vehicle):
            return Vehicle(self.velocity * other.velocity)
        elif isinstance(other, int):
            return Vehicle(self.velocity * other)
        else:
            raise TypeError()
    
    def __rmul__(self, other):
        if isinstance(other, int):
            return Vehicle(other * self.velocity)
        else:
            raise TypeError()
    
    def run(self):
        raise NotImplementedError()


class Car(Vehicle):
    @property
    def run(self):
        print('Run at {}.'.format(self.velocity))

c1 = Car(1)
c2 = Car(2)

print(c1 + c2) # __add__
print(c1 +  2) # __add__
print(3  + c1) # __radd__
print(c1 * c2) # __mul__
print(c1 *  4) # __mul__
print(5  * c1) # __rmul__
print(6  * (c1 + c2) + 7)

Velocity is 3.
Velocity is 3.
Velocity is 4.
Velocity is 2.
Velocity is 4.
Velocity is 5.
Velocity is 25.


### 「ステップ23 パッケージとしてまとめる」より
 - Pythonファイルを**モジュール**と呼び、ディレクトリに複数モジュールを追加したものを**パッケージ**と呼び、１個以上のパッケージをまとめたものを**ライブラリ**と呼ぶ。
   - `from XXX import C`：モジュールXXXにあるクラスCや関数などをインポートする。
   - `import XXX as A`：モジュールXXXをAという名前でインポートできる。
   - `__init__.py`はモジュールをインポートする際に、最初に実行されるファイルである。パッケージにインポートや初期化処理を記述する。空でも構わない。
- `globals`関数でグローバル変数が取得できる。Pythonコマンドとして実行する場合は、`__file__`グローバル変数が定義されている。
 - `sys.path.append`によりモジュールの検索パスに追加できる。
 - キィットキィトで開発するプログラムは次のようなファイル構成にする。
   - `qitqito`ディレクトリにモジュール（Pythonファイル）を置き、`qitqito`パッケージを作る。
   - `tests`に`unittest`用のテストを置き、`tests`パッケージを作る。
     - なお`dezero`では`tests/__init__.py`は無い。その場合もDeZeroパッケージをインストールするか、テストプログラムに`sys.path.append(os.path.join(os.path.dirname(__file__), '..'))`を記述すればモジュールの検索パスに追加してモジュールをインポートできる。
     - しかしエディタにインポートエラーの表示がされてしまう。これを避けるため、キィットキィトでは`tests/__init__.py`を配置する。

```
.
│
├── qitqito
│ ├── __init__.py
│ ├── vehicle.py
│ ├── ...
│ └── ...
│
├── tests
│ ├── __init__.py
│ ├── test_car.py
│ └── ...
│
```

 - キィットキィトの`basic`リポジトリをダウンロード（`clone`）して`basic`ディレクトリに移動すると、上記のファイル構成になっている。

```
$ git clone https://github.com/qitqito/basic.git
$ cd basic
```

 - `python`コマンドに`-m unittest`引数を与えて、テストモードでPythonファイルを実行する。
 - テストファイル（`test*.py`）を指定して実行することもできるし、`discover`サブコマンドの後に指定したディレクトリの全テストファイルをまとめて実行することもできる。
   - 参照：[unittest --- ユニットテストフレームワーク](https://docs.python.org/ja/3/library/unittest.html)

```
$ python -m unittest tests.test_car
```
```
$ python -m unittest discover tests
```

 - キィットキィトの`basic`リポジトリには、`setup.py`もあるため、次のいずれかにより`qitqito`パッケージをインストールできる。
   - `basic`リポジトリをダウンロード（`clone`）して`basic`ディレクトリで`pip install .`
   - `pip install git+https://github.com/qitqito/basic.git`
 - `pip uninstall qitqito`でアンインストールする。